In [2]:
from google.cloud import resourcemanager_v3
# from google.cloud import iam_v1

def collect_iam_data(project_id):
    client = resourcemanager_v3.ProjectsClient()
    iam_policy = client.get_iam_policy(request={"resource": f"projects/{project_id}"})
    
    # Process and return the IAM policy data
    return iam_policy

# Collect data from multiple projects
projects = ["gcphde-prim-dev-data", "gcphde-prim-stage-data", "gcphde-prim-synth-data"]
iam_data = [collect_iam_data(project) for project in projects]

In [3]:
iam_data

[version: 1
 bindings {
   role: "roles/artifactregistry.admin"
   members: "serviceAccount:docker-sa@gcphde-prim-dev-data.iam.gserviceaccount.com"
   members: "user:awsmayanktripathi@gmail.com"
 }
 bindings {
   role: "roles/artifactregistry.repoAdmin"
   members: "serviceAccount:docker-sa@gcphde-prim-dev-data.iam.gserviceaccount.com"
 }
 bindings {
   role: "roles/artifactregistry.serviceAgent"
   members: "serviceAccount:service-50583961506@gcp-sa-artifactregistry.iam.gserviceaccount.com"
 }
 bindings {
   role: "roles/bigquery.admin"
   members: "serviceAccount:sa-bigquery-dbeaver@gcphde-prim-dev-data.iam.gserviceaccount.com"
 }
 bindings {
   role: "roles/bigquery.admin_withcond_230d8d3abb094604c670"
   members: "user:gcpmayanktripathi@gmail.com"
 }
 bindings {
   role: "roles/bigquery.dataEditor"
   members: "serviceAccount:sa-bigquery-dbeaver@gcphde-prim-dev-data.iam.gserviceaccount.com"
 }
 bindings {
   role: "roles/bigquery.dataViewer"
   members: "serviceAccount:sa-bigquery-

In [4]:
from google.cloud import bigquery

def store_iam_data(iam_data):
    client = bigquery.Client()
    table_id = "gcphde090-audit.iam_audit.iam_table"

    # Convert IAM data to appropriate format for BigQuery
    rows_to_insert = [
        {"project_id": data["project"], "policy": str(data["policy"])}
        for data in iam_data
    ]

    errors = client.insert_rows_json(table_id, rows_to_insert)
    if errors == []:
        print("IAM data loaded into BigQuery successfully.")
    else:
        print(f"Errors occurred: {errors}")

store_iam_data(iam_data)

TypeError: 'Policy' object is not subscriptable

In [ ]:
from neo4j import GraphDatabase

def create_graph(iam_data):
    driver = GraphDatabase.driver("bolt://localhost:7687", auth=("neo4j", "password"))

    with driver.session() as session:
        for data in iam_data:
            project_id = data["project"]
            policy = data["policy"]

            # Create project node
            session.run("MERGE (:Project {id: $project_id})", project_id=project_id)

            for binding in policy.bindings:
                role = binding.role
                for member in binding.members:
                    # Create role and member nodes, and relationships
                    session.run("""
                        MATCH (p:Project {id: $project_id})
                        MERGE (r:Role {name: $role})
                        MERGE (m:Member {name: $member})
                        MERGE (m)-[:HAS_ROLE]->(r)
                        MERGE (r)-[:IN_PROJECT]->(p)
                    """, project_id=project_id, role=role, member=member)

    driver.close()

create_graph(iam_data)